import pathlib

import numpy as np
import pandas as pd
from sklearn import linear_model, metrics, model_selection

In [172]:
import pathlib

import numpy as np
import pandas as pd
from sklearn import linear_model, metrics, model_selection
from sklearn import decomposition, ensemble, linear_model, metrics, tree, utils

# Load the data

In [173]:
%%bash

ls /kaggle/input/sdaiakaust-ai-bootcamp-august

README.md
sample-submission.csv
test.csv
train.csv


In [174]:
DATA_DIR = pathlib.Path("/kaggle/input/sdaiakaust-ai-bootcamp-august/")

# training data contains both features and labels
train_df = pd.read_csv(DATA_DIR / "train.csv")

# split training data into features and target
train_features = train_df.drop("labels", axis=1)
train_target = train_df.loc[:, "labels"]

# testing data contains only features
test_features = pd.read_csv(DATA_DIR / "test.csv")

In [175]:
train_features.head()

,p0,p1,p2,p3,p4,p5,p6,p7,p8,p9,...,p1014,p1015,p1016,p1017,p1018,p1019,p1020,p1021,p1022,p1023
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


1024 features

In [176]:
train_target.head()

0    19
1    23
2     9
3    27
4    23
Name: labels, dtype: int64

In [177]:
test_features.head()

,p0,p1,p2,p3,p4,p5,p6,p7,p8,p9,...,p1014,p1015,p1016,p1017,p1018,p1019,p1020,p1021,p1022,p1023
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Define your model

In [256]:
#_seed = 42
#_estimator_hyperpararmeters = {
   # "random_state": np.random.RandomState(_seed),
   # "n_jobs": -1,
#}
#estimator = linear_model.SGDClassifier(**_estimator_hyperpararmeters)
_seed = 42
_classifier_hyperparameters = {
    "criterion": "entropy",
    "max_depth": 6000,
    "max_features": "sqrt",
    "max_samples": 0.999,
    "min_samples_leaf": 1e-3,
    "min_samples_split": 2e-3,
    "n_estimators": 3000,
    "bootstrap": True,
    "max_samples": 0.999,
    "n_jobs": -1,
    "oob_score": True,
    "random_state": np.random.RandomState(_seed)
}
estimator = ensemble.RandomForestClassifier(**_classifier_hyperparameters)

this is the cell to make adjumest to tune hyperparamater and model

# Train your model

In [257]:
_ = estimator.fit(train_features, train_target)

all models use the same fitting code above

# Assess performance on the training data

In [258]:
train_predictions = estimator.predict(train_features)
_report = metrics.classification_report(
    train_target,
    train_predictions
)
print(_report)

              precision    recall  f1-score   support

           1       0.83      0.94      0.88       480
           2       0.77      0.94      0.85       480
           3       0.92      0.88      0.90       480
           4       0.90      0.91      0.90       480
           5       0.90      0.85      0.88       480
           6       0.87      0.87      0.87       480
           7       0.96      0.79      0.86       480
           8       0.81      0.92      0.86       480
           9       0.93      0.82      0.88       480
          10       0.65      0.92      0.76       480
          11       0.85      0.72      0.78       480
          12       0.92      0.88      0.90       480
          13       0.97      0.90      0.93       480
          14       0.92      0.90      0.91       480
          15       0.97      0.89      0.93       480
          16       0.94      0.87      0.91       480
          17       0.96      0.85      0.90       480
          18       0.83    

so we need to know if we are over or underfitting we gotta look at the pred


# Estimate the generalization error using CV

In [259]:
scores = model_selection.cross_val_score(
    estimator,
    train_features,
    train_target,
    cv=5,
    scoring="accuracy",
    n_jobs=-1,
)

/opt/conda/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


this is the code that will show us how well my model is doing with new data

In [260]:
print(f"Estimated validation accuracy: {scores.mean()}")

Estimated validation accuracy: 0.5756696428571428


i should make it 10 precnet higher

# Re-train using the full training data set

In [196]:
_ = estimator.fit(train_features, train_target)

# Make predictions using the test data

In [184]:
test_predictions = estimator.predict(test_features)

# Generate your submission file

In [185]:
_ = (pd.DataFrame
       .from_dict({"Id": test_features.index, "Category": test_predictions})
       .to_csv("submission.csv", index=False))